In [ ]:
from google.colab import drive
import os

# ===============================
# Mount Google Drive
# ===============================
drive.mount("/content/drive")



tokenization

In [ ]:
import pandas as pd
import re
import html
import warnings
import os
import torch

from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# ===============================
# Suppress warnings
# ===============================
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# ===============================
# Paths
# ===============================
DATA_PATH = "/content/drive/MyDrive/NewsSumm Dataset.xlsx"
SAVE_DIR  = "/content/drive/MyDrive/tokenized_primera_data"

TRAIN_DIR = os.path.join(SAVE_DIR, "train")
TEST_DIR  = os.path.join(SAVE_DIR, "test")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# ===============================
# Load dataset
# ===============================
df = pd.read_excel(DATA_PATH)
df = df.dropna(subset=["article_text", "human_summary"])
print("Rows after cleaning:", len(df))

# ===============================
# Resample
# ===============================
df = df.sample(n=10000, random_state=42)
print("Rows after resampling:", len(df))

# ===============================
# Clean text
# ===============================
def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = html.unescape(text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"\+?\d[\d\s\-]{8,}\d", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["article_text"]  = df["article_text"].apply(clean_text)
df["human_summary"] = df["human_summary"].apply(clean_text)

# ===============================
# Train / Test split
# ===============================
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Train rows:", len(train_df))
print("Test rows:", len(test_df))

# ===============================
# Load PRIMERA tokenizer
# ===============================
tokenizer = AutoTokenizer.from_pretrained("allenai/PRIMERA")

# ===============================
# Tokenization settings (SAFE)
# ===============================
MAX_INPUT_LEN = 3072
BATCH_SIZE = 1
MAX_TARGET_LEN = 256

# ===============================
# Batch tokenization function
# ===============================
def tokenize_and_save(articles, summaries, out_dir, name):

    batch_id = 0

    for i in range(0, len(articles), BATCH_SIZE):

        batch_articles = articles[i:i+BATCH_SIZE]
        batch_summaries = summaries[i:i+BATCH_SIZE]

        enc = tokenizer(
            batch_articles,
            truncation=True,
            padding="max_length",
            max_length=MAX_INPUT_LEN,
            return_tensors="pt"
        )

        dec = tokenizer(
            batch_summaries,
            truncation=True,
            padding="max_length",
            max_length=MAX_TARGET_LEN,
            return_tensors="pt"
        )

        labels = dec["input_ids"]
        labels[labels == tokenizer.pad_token_id] = -100

        batch_data = {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": labels
        }

        save_path = os.path.join(out_dir, f"batch_{batch_id}.pt")
        torch.save(batch_data, save_path)

        if batch_id % 20 == 0:
            print(f"{name}: saved batch {batch_id}")

        batch_id += 1

    print(f"{name} tokenization done. Total batches: {batch_id}")

# ===============================
# Run tokenization
# ===============================
tokenize_and_save(
    train_df["article_text"].astype(str).tolist(),
    train_df["human_summary"].astype(str).tolist(),
    TRAIN_DIR,
    "TRAIN"
)

tokenize_and_save(
    test_df["article_text"].astype(str).tolist(),
    test_df["human_summary"].astype(str).tolist(),
    TEST_DIR,
    "TEST"
)

print("\nPRIMERA batch-wise tokenization completed successfully")


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=22cbc520a9484b79193eea689bc8d05fde6dd5d0dbbb713b1619e603764033d6
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


model training

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.optim import AdamW

# --------------------
# Paths
# --------------------
TRAIN_DIR = "/content/drive/MyDrive/tokenized_primera_data/train/"
SAVE_DIR  = "/content/drive/MyDrive/primera_finetuned_model"
MODEL_NAME = "allenai/PRIMERA"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Load model & tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model.to(device)
model.train()
model.config.pad_token_id = tokenizer.pad_token_id

# --------------------
# Optimizer (smaller LR for PRIMERA)
# --------------------
optim = AdamW(model.parameters(), lr=1e-5)

# --------------------
# Load batch files
# --------------------
files = [f for f in os.listdir(TRAIN_DIR) if f.endswith(".pt")]
files.sort()

files = files[:1000]   # train on 1500 batches only

print("Total batches:", len(files))

total_loss = 0.0

# --------------------
# Training loop
# --------------------
for i, fname in enumerate(files, 1):

    batch = torch.load(os.path.join(TRAIN_DIR, fname))

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    optim.zero_grad()

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    # Gradient clipping (important for large models)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optim.step()

    total_loss += loss.item()

    if i % 50 == 0:
        print(f"Batch {i} | Avg Loss: {total_loss / i:.4f}")

    # Free memory
    del batch, input_ids, attention_mask, labels, outputs, loss
    torch.cuda.empty_cache()

# --------------------
# Save model
# --------------------
os.makedirs(SAVE_DIR, exist_ok=True)

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("PRIMERA fine-tuning completed successfully")


model evaluation

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer

# --------------------
# Paths
# --------------------
TEST_DIR  = "/content/drive/MyDrive/tokenized_primera_data/test"
MODEL_DIR = "/content/drive/MyDrive/primera_finetuned_model"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Load model & tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR).to(device)
model.eval()

# --------------------
# ROUGE scorer
# --------------------
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

r1 = r2 = rl = 0.0
r1p = r2p = rlp = 0.0
count = 0

files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".pt")])
files = files[:800]

total_batches = len(files)
print("Total test batches to evaluate:", total_batches)

# --------------------
# Evaluation loop
# --------------------
for idx, fname in enumerate(files, 1):

    if idx % 100 == 0 or idx == 1:
        print(f"Evaluating batch {idx}/{total_batches}")

    batch = torch.load(os.path.join(TEST_DIR, fname))

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    labels = batch["labels"].clone()
    labels[labels == -100] = tokenizer.pad_token_id

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128
        )

    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    refs  = tokenizer.batch_decode(labels, skip_special_tokens=True)

    for pred, ref in zip(preds, refs):
        scores = scorer.score(ref, pred)

        r1  += scores["rouge1"].fmeasure
        r2  += scores["rouge2"].fmeasure
        rl  += scores["rougeL"].fmeasure

        r1p += scores["rouge1"].precision
        r2p += scores["rouge2"].precision
        rlp += scores["rougeL"].precision

        count += 1

    del batch, input_ids, attention_mask, labels, outputs
    torch.cuda.empty_cache()

# --------------------
# Final results
# --------------------
print("\nROUGE RESULTS (PRIMERA)")
print("Samples evaluated:", count)
print("ROUGE-1 F1:", round(r1 / count, 4))
print("ROUGE-2 F1:", round(r2 / count, 4))
print("ROUGE-L F1:", round(rl / count, 4))
print("ROUGE-1 Precision:", round(r1p / count, 4))
print("ROUGE-2 Precision:", round(r2p / count, 4))
print("ROUGE-L Precision:", round(rlp / count, 4))
